# IS5006 Final Project Live Trading Code

## Date : 10th April, 2022

## Group 03 Algo Trading System

## Authors
Jesper Lim Jun Hao (A0248996B) e0934611@u.nus.edu

Pitchappan Pitchappan Ramaswamy (A0236575W) e0740930@u.nus.edu

Seow Kah Yong (A0032747E) e0383424@u.nus.edu

Zhu Yexin (A0083629W) e0696677@u.nus.edu


# Installations and Imports

## Installations
- Alpaca TradeAPI - To interact with a demo market for orders. APIs need to be generated for this
- SpacyTextBlob - Local NLP model for sentiment analysis
- BeutifulSoup4 = Text processing for extracting news from HTML
- Scikit_Fuzzy = For fuzzy logic implementations
- textblob.download_corpora = model used by SpacyTextBlob
- spacy download en_core_web_sm = model used by Spacy

## Imports
- General Python functions for data handling, sending requests, json and multithreading
- Newly installed libraries and models are loaded
- A logger is setup for debugging purposes

## Instalation

In [1]:
%%capture
! pip install alpaca_trade_api
! pip install spacytextblob
! pip install beautifulsoup4
! python -m textblob.download_corpora
! python -m spacy download en_core_web_sm
! pip install scikit-fuzzy

## Libraries

In [2]:
import pandas as pd
from threading import *
import datetime
import time
import numpy as np
import operator
import os
from multiprocessing import *
import json
import requests

print("finished importing normal libraries. beginning fuzzy libs")
import skfuzzy as fuzz
from skfuzzy import control as ctrl
print("finished importing fuzzy. beginning NLP libs")
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob
from bs4 import BeautifulSoup
print("finished importing NLP libs. beginning NLP model load")
nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('spacytextblob')
print("finished importing NLP models")
import alpaca_trade_api as tradeapi
print("imported alpaca lib")

import logging
logging.basicConfig(filename='Logs-Info.log', filemode='w', level=logging.INFO)

finished importing normal libraries. beginning fuzzy libs
finished importing fuzzy. beginning NLP libs
finished importing NLP libs. beginning NLP model load
finished importing NLP models
imported alpaca lib


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Constants

Set constants to run the system

- Symbol = 'BTCUSD' is currently the only supported sysmbl
- tick_time = frequency of operation of the system
- timeframe = frequency of data collection. Make sure it is the same as tick_time for avoiding double orders
- exchange and exchanges = exchange from where we get data to make decisions in ALpaca
- Alpaca API KEY and SECRET KEY = credentials from Alpaca
- SAVE_LOAD_PATH = path in google drive to store results and order history. Give blank string if you'd like to store them in the local runtime
- Limit_Buy_sell_Cost_variance = the price difference we are willling to make for a buy/sell order. we buy at price+this variance and sell at price-this variance. Ideal scenario is where this is 0. But is 0, it takes quite a long time to fill the orders.
- POWERBI_USED = do you need tosend data to a powerBI dashboard
- POWERBI_DATA_URL = PowerBI streaming API endpoint

In [4]:
symbol = "BTCUSD"
tick_time = 60
timeframe = "1Min"
exchanges = ["CBSE"]
exchange = "CBSE"
BASE_URL = "https://paper-api.alpaca.markets"
ALPACA_API_KEY = "PKR055KBNBPSHUVPR494"
ALPACA_SECRET_KEY = "tiAVmsA6dEQOOdIfhY4Sun5wJE5PlU960rPMwL0C"
HISTORY_URL = "https://data.alpaca.markets/v1beta1/crypto"

SAVE_LOAD_PATH = '/content/drive/MyDrive/CUR_IS5006_PATH/Apr10_10AM/'

Limit_Buy_Sell_Cost_variance = 100


POWERBI_USED = False
POWERBI_DATA_URL = 'https://api.powerbi.com/beta/5ba5ef5e-3109-4e77-85bd-cfeb0d347e82/datasets/1fed8683-cbfb-4500-931c-94f543308020/rows?key=VMtMKFOaFpQxeZl5rH058sjm%2F8SSdR7qpvrTUxnAUfLLOhf6OHUCqzTrFNtxMVbauhZYf1QxB3hKuZ0kwyw%2FCA%3D%3D'


# Broker
- Agent to handle all interactions with Alpaca Broker API
- Functions include
    - Getting historical data
    - Getting currnent Ticker Price
    - Getting account details
    - Retrieving Order Status
    - Placing Orders
    - Cancelling Orders

In [5]:
class Broker():

    # Instantiate REST API Connection
    api = tradeapi.REST(key_id=ALPACA_API_KEY, secret_key=ALPACA_SECRET_KEY, 
                        base_url=BASE_URL, api_version='v2')

    @staticmethod
    def get_history(symbol = symbol,timeframe=timeframe,exchanges=exchanges):
        symbol = symbol
        timeframe = timeframe
        exchanges = exchanges

        delta = 0
        if "Day" in timeframe:
            delta = 110
        elif "Min" in timeframe:
            delta = 10
        elif "Hour" in timeframe:
            delta = 10
        else:
            delta = 700
        time = (datetime.datetime.now()- datetime.timedelta(days=delta)).strftime('%Y-%m-%dT%H:%M:%SZ')

        df = Broker.api.get_crypto_bars(symbol=symbol,timeframe=timeframe,start=time,exchanges=exchanges).df
        return df

    @staticmethod
    def get_balance():
        return Broker.api.get_account().cash

    @staticmethod
    def get_crypto_balance():
        positions = Broker.api.list_positions()
        crypto_balances = {}
        for pos in positions:
            crypto_balances[pos.symbol] = pos.qty
        return crypto_balances

    @staticmethod
    def get_account():
        return Broker.api.get_account()

    @staticmethod
    def get_ticker_price(symbol = "BTCUSD", exchanges = ["CBSE"]):
        symbol = symbol
        exchanges = exchanges
        temp_df = Broker.api.get_crypto_bars(symbol=symbol,timeframe="1Min",exchanges=exchanges).df
        if len(temp_df) > 0:
            price = temp_df.iloc[[-1]]['close'].values[0]
        else:
            time = (datetime.datetime.now()- datetime.timedelta(days=3)).strftime('%Y-%m-%dT%H:%M:%SZ')
            temp_df = Broker.api.get_crypto_bars(symbol=symbol,timeframe="1Min",start=time,exchanges=exchanges).df
            price = temp_df.iloc[[-1]]['close'].values[0]
            print("APOCALYPSE AVOIDED")
        return price

    @staticmethod
    def place_market_buy_order(trading_symbol, volume, orderID):
        symbol = trading_symbol
        qty = volume
        side = "buy"
        type_of_transaction = "market"
        Order_ID = orderID
        Broker.api.submit_order(symbol=symbol, qty=qty, side=side, type=type_of_transaction, client_order_id=Order_ID)
        return Broker.get_order_status(Order_ID).id


    @staticmethod
    def place_market_sell_order(trading_symbol, volume, orderID):
        symbol = trading_symbol
        qty = volume
        side = "sell"
        type_of_transaction = "market"
        Order_ID = orderID
        Broker.api.submit_order(symbol=symbol, qty=qty, side=side, type=type_of_transaction, client_order_id=Order_ID)
        return Broker.get_order_status(Order_ID).id


    @staticmethod
    def place_limit_buy_order(trading_symbol, volume, price, orderID):
        symbol = trading_symbol
        qty = volume
        side = "buy"
        type_of_transaction = "limit"
        Order_ID = orderID
        limit_price=price

        Broker.api.submit_order(symbol=symbol, qty=qty, side=side, type=type_of_transaction, limit_price=limit_price, client_order_id=Order_ID)
        return Broker.get_order_status(Order_ID).id

    @staticmethod
    def place_limit_sell_order(trading_symbol, volume, price, orderID):
        symbol = trading_symbol
        qty = volume
        side = "sell"
        type_of_transaction = "limit"
        Order_ID = orderID
        limit_price=price

        Broker.api.submit_order(symbol=symbol, qty=qty, side=side, type=type_of_transaction, limit_price=limit_price, client_order_id=Order_ID)
        return Broker.get_order_status(Order_ID).id

    @staticmethod
    def get_order_status(OrderID):
        return Broker.api.get_order_by_client_order_id(OrderID)

    @staticmethod
    def cancel_order(OrderID):
        Broker.api.cancel_order(OrderID)

    @staticmethod
    def get_orders():
        return Broker.api.list_orders()

    @staticmethod
    def get_positions():
        return Broker.api.list_positions()

    @staticmethod
    def cancel_all_orders():
        return Broker.api.cancel_all_orders()

# Quantitative Agents
Agents that provide Buy and Sell Signals

4 agents have been implemented 
- Simple Moving Average (SMA)
- Exponential Moving Average (EMA)
- Bollinger Bands (BB)
- Bollinger Band Trends (BBT)

FBP is a dummy agent that could be used for future implementations of otther agents

## EMA

In [6]:
class EMA():

    def __init__(self):
        self.signals = [0]
        self.log = logging.getLogger('EMA')
        self.log.info("Created EMA Agent")

    def return_last_signal(self, df):
        
        df['EMA_20'] = df['close'].ewm(span=20, adjust = False).mean()
        df['EMA_50'] = df['close'].ewm(span=50, adjust = False).mean()
        df['signal'] = 0.0
        df['signal'] = np.where(df['EMA_20'] > df['EMA_50'], 1.0, 0.0)
        df['EMA_pos'] = df['signal'].diff()   

        if df.iloc[-1]['EMA_pos']==1:
            self.signals.append(1)
        elif df.iloc[-1]['EMA_pos']==-1:
            self.signals.append(-1)
        else:
            self.signals.append(0)

        self.log.info(('signal',self.signals[-1]))

        return self.signals[-1]

    def __str__(self):
        return 'EMA'

## SMA

In [7]:
class SMA():

    def __init__(self):
        self.signals = [0]
        self.log = logging.getLogger('SMA')
        self.log.info("Created SMA Agent")

    def return_last_signal(self, df):

        df['SMA_20'] = df['close'].rolling(window=20).mean()
        df['SMA_50'] = df['close'].rolling(window=50).mean()
        df['signal'] = 0.0
        df['signal'] = np.where(df['SMA_20'] > df['SMA_50'], 1.0, 0.0)
        df['SMA_pos'] = df['signal'].diff()   

        if df.iloc[-1]['SMA_pos']==1:
            self.signals.append(1)
        elif df.iloc[-1]['SMA_pos']==-1:
            self.signals.append(-1)
        else:
            self.signals.append(0)

        self.log.info(('signal',self.signals[-1]))

        return self.signals[-1]

    def __str__(self):
        return 'SMA'

## BB

In [8]:
class BB():

    def __init__(self):
        self.signals = [0]
        self.log = logging.getLogger('BB')
        self.log.info("Created BB Agent")

    def return_last_signal(self, df):

        df['SMA_20'] = df['close'].rolling(window=20).mean()
        df['STD_DEV_20'] = df['close'].rolling(window=20).std()
        df['BB_Low_20'] = df['SMA_20'] - df['STD_DEV_20'] * 2
        df['BB_High_20'] = df['SMA_20'] + df['STD_DEV_20'] * 2
        
        df['cross_high'] = np.where(df['close'] > df['BB_High_20'], 1, 0)
        df['sell'] = df['cross_high'].diff()
        
        df['cross_low'] = np.where(df['close'] < df['BB_Low_20'], 1, 0)
        df['buy'] = df['cross_low'].diff()
        
        if df.iloc[-1]['buy']==-1:
            self.signals.append(1)
        elif df.iloc[-1]['sell']==-1:
            self.signals.append(-1)
        else:
            self.signals.append(0)

        self.log.info(('signal',self.signals[-1]))

        return self.signals[-1]

    def __str__(self):
        return 'BB'

## BBTrend

In [9]:
class BBT():

    def __init__(self):
        self.signals = [0]
        self.last_action = 0
        self.log = logging.getLogger('BBTrend')
        self.log.info("Created BBTrend Agent")

    def return_last_signal(self, df):

        df['SMA_20'] = df['close'].rolling(window=20).mean()
        df['STD_DEV_20'] = df['close'].rolling(window=20).std()
        df['BB_Low_20'] = df['SMA_20'] - df['STD_DEV_20'] * 2
        df['BB_High_20'] = df['SMA_20'] + df['STD_DEV_20'] * 2

        df['SMA_50'] = df['close'].rolling(window=50).mean()
        df['STD_DEV_50'] = df['close'].rolling(window=50).std()
        df['BB_Low_50'] = df['SMA_50'] - df['STD_DEV_50'] * 2
        df['BB_High_50'] = df['SMA_50'] + df['STD_DEV_50'] * 2

        df['LowBands'] = abs(df['BB_Low_20'] - df['BB_Low_50'])
        df['HighBands'] = abs(df['BB_High_20'] - df['BB_High_50'])
        df['BBT'] = (df['LowBands'] - df['HighBands']) / df['SMA_20']

        if (df.iloc[-1]['BBT'] > 0) and (self.last_action != 1):
            self.signals.append(1)
            self.last_action = 1
        elif (df.iloc[-1]['BBT'] < 0) and (self.last_action != -1):
            self.signals.append(-1)
            self.last_action = -1
        else:
            self.signals.append(0)

        self.log.info(('signal',self.signals[-1]))
        
        return [self.signals[-1],df.iloc[-1]['BBT']]

    def __str__(self):
        return 'BBT'

## FBProphet

In [10]:
class FBP():

    def __init__(self):
        self.signals = [0]
        self.log = logging.getLogger('FBProphet')
        self.log.info("Created FBProphet Agent")

    def return_last_signal(self, df):
        self.log.info(('signal',0))
        return 0

    def __str__(self):
        return 'FBP'

# Signals and History
Coordinate all Signal agents and prepare them constantly for sending to CEO whenever requested

In [11]:
class Signals():

    def __init__(self, SMA, EMA, BB, BBT, FBP):
        self.lock = Lock()
        self.tick_time = tick_time

        self.symbol = symbol
        self.timeframe = timeframe
        self.exchange = exchange

        self.log = logging.getLogger('Signals')
        self.log.info("Initialized Signals")

        if (os.path.isfile(SAVE_LOAD_PATH+'MarketHistory.csv')):
            self.market_history = pd.read_csv(SAVE_LOAD_PATH+'MarketHistory.csv',index_col=0)
            self.market_history.set_index('time')
            self.log.info('Past Market History Loaded')
        else:
            self.market_history = pd.DataFrame()
            self.log.info('No Past MarketHistory found. Creating new dataframe')
    
        self.SMA = SMA
        self.EMA = EMA
        self.BB = BB
        self.BBT = BBT
        self.FBP = FBP

        self.thread = Thread(name=self.__str__(), target=self.loop)
        self.thread.start()

    def __str__(self):
        return 'Signals'

    def loop(self):
        while True:
            self.tick()
            time.sleep(self.tick_time)

    def tick(self):
        self.lock.acquire()
        self.log.info("LOCK ACQUIRED")

        df_real = Broker.get_history(symbol = self.symbol,timeframe=self.timeframe,exchanges=[self.exchange])
        df = df_real.copy()
        SMA_signal = self.SMA.return_last_signal(df)
        EMA_signal = self.EMA.return_last_signal(df)
        BB_signal = self.BB.return_last_signal(df)
        [BBT_signal,BBT_value] = self.BBT.return_last_signal(df)
        FBP_signal = self.FBP.return_last_signal(df)

        if (os.path.isfile(SAVE_LOAD_PATH+'MarketHistory.csv')):
            # print("BBT SIGNAL CHECK : ",np.array(self.market_history.BBT)[np.nonzero(np.array(self.market_history.BBT))[0][-1]], BBT_signal)
            if np.array(self.market_history.BBT)[np.nonzero(np.array(self.market_history.BBT))[0][-1]] == BBT_signal:
                BBT_signal = 0

        last_row = df_real.iloc[[-1]].copy()

        last_row['SMA'], last_row['EMA'], last_row['BB'], last_row['BBT'], last_row['FBP'] = [SMA_signal,EMA_signal,BB_signal,BBT_signal,FBP_signal]
        last_row['BBT_val'] = [BBT_value]
        last_row['time'] = last_row.index
        time = last_row.index.strftime('%Y-%m-%d %H:%M:%S+00:00')[0]
        self.log.info((time,SMA_signal,EMA_signal,BB_signal,BBT_signal,FBP_signal))

        self.market_history = pd.concat([self.market_history, last_row], axis=0)

        self.market_history = self.market_history[['time', 'exchange',
                                    'open', 'high', 'low', 'close', 'volume', 
                                    'trade_count', 'vwap', 'SMA',
                                    'EMA', 'BB', 'BBT', 'FBP', 'BBT_val']]
        self.market_history = self.market_history.drop_duplicates(subset='time', keep="first")
        self.log.info('Saving Market History')
        self.market_history.to_csv(SAVE_LOAD_PATH+'MarketHistory.csv')
        self.log.info("RELEASING LOCK")
        self.lock.release()

    def peek(self):
        row = self.market_history.iloc[-1]
        self.log.info('Peeking Latest Signals')
        return row
    
    def dump_market_history(self):
        df = self.market_history.drop_duplicates(subset='time', keep="first")
        pd.set_option('display.max_rows', None)
        pd.set_option('display.max_columns',None)
        self.log.info('Dumping Market History')
        self.log.info(df)
        return df

In [12]:
# S = Signals(SMA(),EMA(),BB(),BBT(),FBP())
# for i in [1,2,3,4,5,6,7]:
#     print(i)
#     print(S.peek())
#     time.sleep(10)
# S.dump_market_history()

# Orders
Agent to handle orders, analyse results and store them.

Data handles
- Order Table
    - Table of all orders placed
    - Monitor Limit orders and close them whenever they are finished
    - Cancel orders if they take too long to fill (Current is 100 ticks)
    - Save the results to a CSV periodically
- Order Profitability Table
    - Analyse all closed orders to judge if an order is profitable
    - Order Profitability is judge by analyzing market trend after an order was placed
        - If the market trend is still increasing after a buy, we consider our buy to be a good buy. If it dropped below zero, we consider it to be a bad buy.
        - We monitor the trend for 50 ticks after each action to get a better idea of the profitability of a trade
    - Saves these results to a table periodically
- Wallet Table
    - Contains an up-to-date table containing our crypto holdings and cash along with PnL so far

Operations
- Tick
    - Executed every tick_time (1 minute) to update all order statuses, update order profitability and wallet. Saves the files every tick.
- Place order
    - Places an order with the broker and begins monitoring the order
- Extra functions to log and dump local varaibles whenever requested


In [13]:
class Orders():
    def __init__(self, signals):
        self.lock = Lock()
        self.tick_time = tick_time
        self.log = logging.getLogger('Orders')
        self.log.info("Initialized Orders")
        self.signals = signals

        if (os.path.isfile(SAVE_LOAD_PATH+'Orders.csv')):
            self.orders = pd.read_csv(SAVE_LOAD_PATH+'Orders.csv',index_col=0)
            self.log.info("Loading Orders")
        else:
            self.orders = pd.DataFrame(columns = ['Order_ID', 'Client_Order_ID', 'action', 'target_price', 'filled_price', 
                                                  'EMA', 'SMA', 'BB', 'BBT', 'FBP', 'BBT_val',                                                 
                                                  'target_volume', 'filled_volume', 'order_status', 'stop_loss', 'take_profit', 'total_ticks'])
            self.log.info("No Past Orders")   

        if (os.path.isfile(SAVE_LOAD_PATH+'OrderProfitability.csv')):
            self.order_profits = pd.read_csv(SAVE_LOAD_PATH+'OrderProfitability.csv',index_col=0)
            self.log.info("Loading Order Profits")
        else:
            self.order_profits = pd.DataFrame(columns = ['Order_ID', 'Client_Order_ID', 'action', 'EMA', 'SMA', 'BB', 'BBT', 
                                                        'FBP', 'BBT_val', 'price', 'total_ticks', 'profit_ticks_5'])
            self.log.info("No Past Order Profits")  
        
        if (os.path.isfile(SAVE_LOAD_PATH+'Wallet.csv')):
            self.wallet = pd.read_csv(SAVE_LOAD_PATH+'Wallet.csv',index_col=0)
            self.log.info("Loading Wallet")
        else:
            self.wallet = pd.DataFrame(columns = ['time', 'cash', 'crypto', 
                                                  'crypto_value', 'num_open_orders', 'num_orders', 'total_assets', 'PnL','crypto_price'])
            self.log.info("No Past Wallet")  

        self.order_count = len(self.orders)
        self.order_profits_count = len(self.order_profits)
        self.wallet_count = len(self.wallet)

        self.thread = Thread(name=self.__str__(), target=self.loop)
        self.thread.start()

    def loop(self):
        while True:
            self.tick()
            time.sleep(self.tick_time)
										
    def tick(self):
        self.lock.acquire()
        self.log.info("TICK LOCK ACQUIRED")
        self.log.info("OrderTick")

        self.order_count = len(self.orders)
        self.order_profits_count = len(self.order_profits)
        self.wallet_count = len(self.wallet)

        self.orders = pd.DataFrame(self.orders)
        self.order_profits = pd.DataFrame(self.order_profits)
        self.wallet = pd.DataFrame(self.wallet)


        self.log.info("Checking StopLoss and TakeProfit")
        # TakeProfit, StopLoss
        for i in range(len(self.orders)):
            order_details = self.orders.iloc[i]
            current_price = float(Broker.get_ticker_price())

            if order_details['action'] == 'buy':
                if order_details['stop_loss'] >= current_price:
                    self.log.info("STOP LOSS FOUND")
                    Broker.cancel_order(order_details['Order_ID'])
                    # Market Sell. No records in tradebook needed. will reflect in wallet.
                    Broker.place_market_sell_order(symbol, order_details['filled_volume'], str(time.time()))
                if order_details['take_profit'] <= current_price:
                    Broker.cancel_order(order_details['Order_ID'])
                    self.log.info("TAKE PROFIT FOUND")
                    # Market Sell. No records in tradebook needed. will reflect in wallet.
                    Broker.place_market_sell_order(symbol, order_details['filled_volume'], str(time.time()))
        
        self.log.info("Cancelling Long Orders")
        # Cancel Orders that are over 100 ticks
        for i in range(len(self.orders)):
            order_details = self.orders.iloc[i]
            if order_details['total_ticks'] > 100 and order_details['order_status'] != 'closed':
                self.cancel_order(order_details['Client_Order_ID'])

        self.order_count = len(self.orders)
        self.order_profits_count = len(self.order_profits)
        self.wallet_count = len(self.wallet)

        self.log.info("Update Orders")
        self.log.info(len(self.orders))
        # self.log.info(self.orders)
        # Update Orders
        for i in range(len(self.orders)):
            # self.log.info(("updating ",i))
            self.orders.at[i,'total_ticks'] = self.orders.at[i,'total_ticks'] + 1
            order_details = self.orders.iloc[i]
            # self.log.info("checking status")
            if order_details['order_status'] != 'closed':
                # self.log.info('updating open order')
                status = Broker.get_order_status(order_details['Client_Order_ID'])
                # self.log.info("got status of open order")
                filled_price = status.filled_avg_price
                if filled_price == None:
                    filled_price = 0
                filled_qty = status.filled_qty
                if filled_qty == None:
                    filled_qty = 0
                self.orders.at[i,'filled_price'] = float(filled_price)
                self.orders.at[i,'filled_volume'] = float(filled_qty)
                
                if self.orders.at[i,'filled_volume'] == order_details['target_volume']:
                    self.log.info("CLosing order as filled")
                    self.orders.at[i,'order_status'] = 'closed'    
                elif float(status.filled_qty) > 0:
                    self.log.info("Partial Fill Order found")
                    self.orders.at[i,'order_status'] = 'partial'
                else:
                     self.orders.at[i,'order_status'] = 'empty'
                    
                if status.canceled_at is not None:
                    self.orders.at[i,'order_status'] = 'closed'

        self.log.info("Update Order Profitability")
        self.log.info(len(self.order_profits))
        # Update existing tickers
        self.log.info("BEGIN FOR LOOP")
        for i in range(len(self.order_profits)):
            # self.log.info("START TICK UPDATE")
            self.log.info(("updating ticks for ",i+1,"out of ",len(self.order_profits)))
            self.order_profits.at[i,'total_ticks'] = self.order_profits.at[i,'total_ticks'] + 1

            BBT_value = self.signals.peek()['BBT_val']

            if self.order_profits.at[i,'action'] == 'buy':
                if self.order_profits.at[i,'total_ticks'] <= 50:
                    self.log.info(['BUY','Cur BBT',BBT_value,'order_BBT',float(self.order_profits.at[i,'BBT_val'])])
                    if BBT_value >= float(self.order_profits.at[i,'BBT_val']):
                        self.order_profits.at[i,'profit_ticks_5'] = self.order_profits.at[i,'profit_ticks_5'] + 1
                    elif BBT_value < 0:
                        self.order_profits.at[i,'profit_ticks_5'] = self.order_profits.at[i,'profit_ticks_5'] - 1

            elif self.order_profits.at[i,'action'] == 'sell':
                if self.order_profits.at[i,'total_ticks'] <= 50:
                    # price = Broker.get_ticker_price()
                    # self.log.info([float(price), float(self.order_profits.at[i,'price'])])
                    self.log.info(['SELL','Cur BBT',BBT_value,'order_BBT',float(self.order_profits.at[i,'BBT_val'])])
                    if BBT_value <= float(self.order_profits.at[i,'BBT_val']):
                        self.order_profits.at[i,'profit_ticks_5'] = self.order_profits.at[i,'profit_ticks_5'] + 1
                    elif BBT_value > 0:
                        self.order_profits.at[i,'profit_ticks_5'] = self.order_profits.at[i,'profit_ticks_5'] - 1

            elif self.order_profits.at[i,'action'] == 'hold':
                if self.order_profits.at[i,'total_ticks'] <= 50:
                    self.order_profits.at[i,'profit_ticks_5'] = self.order_profits.at[i,'profit_ticks_5'] + 0

        
        self.log.info("Add new closed trades to analysis")
        # get list of all closed order ids and add them
        closed_order_ids = list(self.orders.loc[self.orders['order_status'] == "closed"]['Order_ID'])
        recorded_order_ids = list(self.order_profits['Order_ID'])
        to_add_order_ids = list(set(closed_order_ids) - set(recorded_order_ids))
        for order_to_add in to_add_order_ids:
            self.log.info("ADDING ORDER TO PROFIT ANALYSIS")
            self.log.info(order_to_add)
            row = list(self.orders.loc[self.orders['Order_ID'] == order_to_add][['Order_ID', 'Client_Order_ID', 'action', 'EMA', 'SMA', 'BB', 'BBT', 
                                                        'FBP', 'BBT_val', 'filled_price']].values[0])
            
            row.append(0) # pnl
            row.append(0) # ticks
            
            self.log.info("Order Profits updated")
            self.order_profits.loc[len(self.order_profits)] = row
        
        self.log.info("Wallet Update")
        # Add Wallet Line
        time = datetime.datetime.now().strftime('%Y-%m-%dT%H:%M:%SZ')
        self.log.info(time)
        acc_dets = Broker.get_account()
        cash = float(acc_dets.cash)
        total_assets = float(acc_dets.equity)
        self.log.info("Find Cryptos")
        crypto_bals = Broker.get_crypto_balance()
        crypto = 0
        if len(crypto_bals) > 0:
            crypto = crypto_bals[symbol]
        # self.log.info("Find Ticker")
        crypto_price = Broker.get_ticker_price()
        crypto_val = float(total_assets - cash)
        # self.log.info("Find Open Orders from Broker")
        num_open_orders = len(Broker.api.list_orders(status='open'))
        # self.log.info("Find All Orders from Broker")
        num_orders = len(Broker.api.list_orders(status='all'))
        # self.log.info("Wallet Details ready")
        pnl = total_assets - 100000
        # self.log.info("Adding entry to wallet")
        self.log.info(([time,cash,crypto,crypto_val,num_open_orders,num_orders,total_assets,pnl,crypto_price]))
        self.wallet.loc[len(self.wallet)] = [time,cash,crypto,crypto_val,num_open_orders,num_orders,total_assets,pnl,crypto_price]

        
        # self.log(('wallet entry',[time,cash,crypto,crypto_val,num_open_orders,num_orders,total_assets,pnl,crypto_price]))

        # Save Files
        self.log.info("Save Files")
        self.log.info('Saving Orders')
        self.log.info(len(self.orders))
        self.orders.to_csv(SAVE_LOAD_PATH+'Orders.csv')
        self.log.info('Saving Order Profits')
        self.log.info(len(self.order_profits))
        self.order_profits.to_csv(SAVE_LOAD_PATH+'OrderProfitability.csv')
        self.log.info('Saving Wallet')
        self.wallet.to_csv(SAVE_LOAD_PATH+'wallet.csv')

        self.log.info("TICK LOCK RELEASED")
        self.lock.release()
        

    def place_order(self,action, order_type, volume, price, stop_loss, take_profit, EMA, SMA, BB, BBT, FBP, BBT_val):

        self.lock.acquire()
        self.log.info("PLACE ORDER LOCK ACQUIRED")
        
        Client_Order_ID = str(time.time())
        action = action
        order_type = order_type
        target_volume = volume
        target_price = price
        stop_loss = stop_loss
        take_profit = take_profit
        EMA = EMA
        SMA = SMA
        BB = BB
        BBT = BBT
        FBP = FBP
        BBT_val = BBT_val

        order_type = order_type

        filled_price = 0
        filled_volume = 0
        total_ticks = 0
        order_status = 'open'

        Order_ID = 0
        self.log.info(["Placing Order with type ",order_type])
        if order_type == 'limit':
            if action == 'buy':
                Order_ID = Broker.place_limit_buy_order(symbol, target_volume, target_price, Client_Order_ID)
            elif action =='sell':
                Order_ID = Broker.place_limit_sell_order(symbol, target_volume, target_price, Client_Order_ID)
        else:
            if action == 'buy':
                Order_ID = Broker.place_market_buy_order(symbol, target_volume, Client_Order_ID)
            elif action =='sell':
                Order_ID = Broker.place_market_sell_order(symbol, target_volume, Client_Order_ID)
                
        print("PLACING ORDER WITH ",action, symbol, target_volume, target_price, Client_Order_ID)
        print("B4 order ",len(self.orders))

        self.orders.loc[len(self.orders)] = [Order_ID, Client_Order_ID, action, target_price, filled_price, 
                                             EMA, SMA, BB, BBT, FBP, BBT_val, target_volume, filled_volume, 
                                             order_status, stop_loss, take_profit, total_ticks]
        print("After order ",len(self.orders))
        # self.log.info("dumping orders")
        # self.log.info(self.orders)
        self.log.info("PLACE ORDER LOCK RELEASED")
        self.lock.release()

    def dump_order_profitability(self):
        return self.order_profits

    def dump_orders(self):
        return self.orders

    def dump_wallet(self):
        return self.wallet

    def __str__(self):
        return 'Orders'

# Knwoledge
Contains Agent Weights

Agent Weights are used to judge how good an agent is for a particular signal

If SMA has a buy weight of 10, hod weight of 0.1 and a sell weight of 1, it means SMA is a good agent to follow when it says we should  buy but a bad choice otherwise

The class saves the agent weights to a CSV periodically (every 10 ticks)

It also contains helper functions to retrieve agent weights when evaluating signals

In [14]:
class Knowledge():

    def __init__(self):
        self.lock = Lock()
        self.tick_time = tick_time * 10
        self.log = logging.getLogger('Knowledge')
        self.log.info("Initialized Knowledge")

        if (os.path.isfile(SAVE_LOAD_PATH+'AgentWeights.csv')):
            temp_dict = pd.read_csv(SAVE_LOAD_PATH+'AgentWeights.csv',index_col=0).to_dict()
            agents = ['BB','BBT','SMA','EMA','FBP']
            action = 'action'

            new_dict = {}
            for agent in agents:
                # print(agent)
                sub_dict = {}
                for j in temp_dict[agent].keys():
                    key = temp_dict[action][j]
                    # print(key)
                    val = temp_dict[agent][j]
                    sub_dict[key] = val
                new_dict[agent] = sub_dict
            
            self.agent_weights = new_dict
            self.log.info("Loading Knowledge of Signals")
        else:
            self.agent_weights = {
                'BB':{1:1,0:1,-1:1}, 
                'BBT':{1:1,0:1,-1:1},
                'EMA':{1:1,0:1,-1:1}, 
                'SMA':{1:1,0:1,-1:1}, 
                'FBP':{1:0,0:0,-1:0} 
                }
            self.log.info("No Knowledge. Setting all to weights to 1")    
            # BB :  Buy = 10, Sell = 1
            # BBT : Buy = -3, Sell = -3
            # Final Weight : 0.1 - 0.05BTC. if its 1-3 - 0.1 BTC. 3-5 - 0.3 BTC. >5 - 0.3 BTC

        self.thread = Thread(name=self.__str__(), target=self.loop)
        self.thread.start()

    def loop(self):
        while True:
            self.tick()
            time.sleep(self.tick_time)
										
    def tick(self):
        self.lock.acquire()
        self.log.info("TICK LOCK ACQUIRED")
        self.log.info('Saving Agent Weights')
        if (os.path.isfile(SAVE_LOAD_PATH+'AgentWeights.csv')):
            df_agent_weights = pd.DataFrame.from_dict(self.agent_weights)
            df_agent_weights['action'] = df_agent_weights.index
            df_agent_weights.reset_index(inplace=True, drop=True)
            df_agent_weights.to_csv(SAVE_LOAD_PATH+'AgentWeights.csv')
        else:
            df_agent_weights = pd.DataFrame.from_dict(self.agent_weights)
            df_agent_weights['action'] = df_agent_weights.index
            df_agent_weights.reset_index(inplace=True, drop=True)
            df_agent_weights.to_csv(SAVE_LOAD_PATH+'AgentWeights.csv')
        self.log.info("TICK LOCK RELEASED")
        self.lock.release()

    def signal_importance(self, agent, signal):
        # imp = 0
        # if type(list(self.agent_weights[agent].keys())[0]) == int:
        #     imp = self.agent_weights[agent][signal]
        # else:
        # print(self.agent_weights)
        imp = self.agent_weights[agent][signal]
        return imp

    def update_signal_weight(self, agent, weight):
        self.lock.acquire()
        self.log.info("UPDATE LOCK ACQUIRED")
        self.agent_weights[agent] = weight
        self.log.info(("Updated weight of",agent,weight))
        self.log.info("UPDATE LOCK RELEASED")
        self.lock.release()

    def dump_agent_weights(self):
        pd.set_option('display.max_rows', None)
        pd.set_option('display.max_columns',None)
        self.log.info(pd.DataFrame.from_dict(self.agent_weights))
        return pd.DataFrame.from_dict(self.agent_weights)

    def __str__(self):
        return 'Knowledge'

# Learner

Periodically uses Orders and OrderProfits Tables from the Order Class to update and tweak the weights of all agents.

Learning happens every 10 ticks.

Once the learning process has begun. 
- Say if we are updating the weights for EMA.
- We find all the buy actions we did and make a count of times EMA supported or resisted the action.
- If EMA supported said buy and it was a good trade, we call that a good buy.
- If EMA said buy and it wa a bad trade, we call that a bad buy.
- we then find the total buys and update the buy weight of EMA as follows
    - new_weights = old_weight * (good_buy-bad_buy)/total_buy
- weights are capped at 1 for the maximum and 0.01 for the minimum value.
- If we dont set such limits, the values sometimes blow up.
- Resetting weights can sometimes lead to better performance of the system but such a method hasn;t been implemented or explored in detail. 


In [15]:
class Learner():

    def __init__(self, knowledge, order):
        self.lock = Lock()
        self.tick_time = tick_time * 10
        self.log = logging.getLogger('Learner')
        self.log.info("Initialized Learner")
        self.knowledge = knowledge
        self.order = order

        self.thread = Thread(name=self.__str__(), target=self.loop)
        self.thread.start()

    def loop(self):
        while True:
            self.tick()
            time.sleep(self.tick_time)
										
    def tick(self):
        self.lock.acquire()
        self.log.info('Beginning a learning experience')
        agent_weights = pd.DataFrame(self.knowledge.dump_agent_weights())
        order_profits = pd.DataFrame(self.order.dump_order_profitability())
        order_profits = order_profits.loc[order_profits['total_ticks'] > 5]
        agents = ['SMA','EMA','BB','BBT','FBP']
        for i in agents:
            self.log.info(("LEARNING FOR ",i))
            # self.log.info(order_profits[i])
            good_buy = len(order_profits[(order_profits[i]==1) & (order_profits['profit_ticks_5']>0)])
            # self.log.info(good_buy)
            bad_buy = len(order_profits[(order_profits[i]==1) & (order_profits['profit_ticks_5']<0)])
            # self.log.info(bad_buy)
            diff_buy = good_buy-bad_buy
            buy = len(order_profits[(order_profits[i]==1)]) + 1
            # self.log.info(buy)
            buy_wt = self.knowledge.signal_importance(i, 1)
            # self.log.info(buy_wt)
            new_buy_wt = buy_wt * (1+(diff_buy/buy))
            if abs(new_buy_wt) > 10:
                new_buy_wt = 10 * new_buy_wt / abs(new_buy_wt)
            if abs(new_buy_wt) < 0.01 and abs(new_buy_wt) != 0:
                new_buy_wt = 0.01 * new_buy_wt / abs(new_buy_wt)
            self.log.info(("BUY",i,good_buy,bad_buy,diff_buy))

            good_sell = len(order_profits[(order_profits[i]==-1) & (order_profits['profit_ticks_5']>0)])
            bad_sell = len(order_profits[(order_profits[i]==-1) & (order_profits['profit_ticks_5']<0)])
            diff_sell = good_sell-bad_sell
            sell = len(order_profits[(order_profits[i]==-1)]) + 1
            sell_wt = self.knowledge.signal_importance(i, -1)
            new_sell_wt = sell_wt * (1+(diff_sell/sell))
            if abs(new_sell_wt) > 10:
                new_sell_wt = 10 * new_sell_wt / abs(new_sell_wt)
            if abs(new_sell_wt) < 0.01 and abs(new_sell_wt) != 0:
                new_sell_wt = 0.01 * new_sell_wt / abs(new_sell_wt)
            self.log.info(("SELL",i,good_sell,bad_sell,diff_sell))

            good_hold = len(order_profits[(order_profits[i]==0) & (order_profits['profit_ticks_5']<0)])
            bad_hold = len(order_profits[(order_profits[i]==0) & (order_profits['profit_ticks_5']>0)])
            diff_hold = good_hold-bad_hold
            hold = len(order_profits[(order_profits[i]==0)]) + 1
            hold_wt = self.knowledge.signal_importance(i, 0)
            new_hold_wt = hold_wt * (1+(diff_hold/hold))
            if abs(new_hold_wt) > 10:
                new_hold_wt = 10 * new_hold_wt / abs(new_hold_wt)
            if abs(new_hold_wt) < 0.01 and abs(new_hold_wt)!=0:
                new_hold_wt = 0.01 * new_hold_wt / abs(new_hold_wt)
            self.log.info(("HOLD",i,good_hold,bad_hold,diff_hold))

            new_wt_dict = {-1:new_sell_wt,0:new_hold_wt,1:new_buy_wt}
            self.knowledge.update_signal_weight(i, new_wt_dict)

            self.log.info(("updating weight for ",i,new_wt_dict))
        self.log.info('learning cycle finished')

        self.lock.release()

    def __str__(self):
        return 'Learner'

# News Reviewer
Collects news from Benzinga, a financial news site and performs sentiment analysis and fuzzy logic to give us a market outlook.

- News of the past 12/24 hrs is collected every 30 ticks (30 minutes)
- NLP from Spacy and TextBlob is applied to all news articles to identify positive and negative news articles
    - Each Fuzzy Logic Results gives us 2 values
        - Polarity 
            - +1 to -1
            - how positive or negative is the news
        - Subjectivity
            - 0 to 1
            - how subjective is the news or is it just rambling
- Once we have all polarities and subjectivities, we use Fuzzy Logic to convert the values to strength of a news using the following guidelines
    - Rules for Okay News
        - Rule 1 : If news article was casual, then its okay
        - Rule 2 : If news article was neutral and normal, then its okay
        - Rule 3 : If news article was neutral and serious, then its okay
    - Rules for Bad News
        - Rule 4 : If news article is negative and normal, its bad news
        - Rule 5 : If news article is negative and serious, its bad news
    - Rules for Good News
        - Rule 6 : If news article is positie and normal, its good news
        - Rule 7 : If news article is positive and serious, its good news
- Strength of a news is measured from -50 to 50
- We convert strength into news outlooks
- We then perform an exponential average of all news article news outlook with the latest news having greatest importance and send the result as overall news outlook whenever the CEO requests

In [16]:
class News_Reviewer():

    def __init__(self):
        self.lock = Lock()
        self.tick_time = tick_time * 30
        self.log = logging.getLogger('News')
        self.log.info("Initialized News")
        self.current_strength = 0

        self.thread = Thread(name=self.__str__(), target=self.loop)
        self.thread.start()


    def News_Fuzzy(self, avg_Polarity, avg_Subjectivity):
        # Fuzzy Logic

        # Value of Score is between -1 and 1 with 0.01 space for steps between them in the universe
        Polarity = ctrl.Antecedent(np.arange(-1, 1.1, 0.01), 'Polarity')

        # Value of Magnitude is between -1 and 1 with 0.01 space for steps between them in the universe
        Subjectivity = ctrl.Antecedent(np.arange(0, 1, 0.01), 'Subjectivity')

        # Value of Strength is between 0 and 100 with 0.5 space for steps between them in the universe
        strength = ctrl.Consequent(np.arange(-50, 50.5, 0.5), 'strength')

        # Automf = 3
        # dismal, poor, mediocre, average (always middle), decent, good, excellent = if set to 7. 
        # Ref Link : https://github.com/scikit-fuzzy/scikit-fuzzy/blob/master/skfuzzy/control/fuzzyvariable.py
        # Polarity.automf(3) # Tweet was negative, neutral or positive
        # Subjectivity.automf(3) # Tweet was casual, noticeable or serious
        # strength.automf(3)  # Tweet was bad, okay or good

        Polarity['negative'] = fuzz.trimf(Polarity.universe, [-1, -1, 0])
        Polarity['neutral'] = fuzz.trimf(Polarity.universe, [-0.05, 0, 0.05])
        Polarity['positive'] = fuzz.trimf(Polarity.universe, [0, 1, 1])

        Subjectivity['casual'] = fuzz.trimf(Subjectivity.universe, [0, 0, 0.1])
        Subjectivity['normal'] = fuzz.trimf(Subjectivity.universe, [0.08, 0.15, 0.3])
        Subjectivity['serious'] = fuzz.trimf(Subjectivity.universe, [0.2, 1, 1])

        strength['bad'] = fuzz.trimf(strength.universe, [-50, -50, 0])
        strength['okay'] = fuzz.trimf(strength.universe, [-10, 0, 10])
        strength['good'] = fuzz.trimf(strength.universe, [0, 50, 50])

        # antecedents: Polarity, Subjectivity
        # consequent: strength      
        # Rules

        # Rules for Okay News
        # Rule 1 : If news article was casual, then its okay
        rule1 = ctrl.Rule(antecedent=(Subjectivity['casual']), consequent=strength['okay'], label='rule1')

        # Rule 2 : If news article was neutral and normal, then its okay
        rule2 = ctrl.Rule(antecedent=(Polarity['neutral'] & Subjectivity['normal']), consequent=strength['okay'], label='rule2')

        # Rule 3 : If news article was neutral and serious, then its okay
        rule3 = ctrl.Rule(antecedent=(Polarity['neutral'] & Subjectivity['serious']), consequent=strength['okay'], label='rule3')

        # Rules for Bad News
        # Rule 4 : If news article is negative and normal, its bad news
        rule4 = ctrl.Rule(antecedent=(Polarity['negative'] & Subjectivity['normal']), consequent=strength['bad'], label='rule4')

        # Rule 5 : If news article is negative and serious, its bad news
        rule5 = ctrl.Rule(antecedent=(Polarity['negative'] & Subjectivity['serious']), consequent=strength['bad'], label='rule5')

        # Rules for Good News
        # Rule 6 : If news article is positie and normal, its good news
        rule6 = ctrl.Rule(antecedent=(Polarity['positive'] & Subjectivity['normal']), consequent=strength['good'], label='rule6')

        # Rule 7 : If news article is positive and serious, its good news
        rule7 = ctrl.Rule(antecedent=(Polarity['positive'] & Subjectivity['serious']), consequent=strength['good'], label='rule7')

        # Create the Fuzzy System
        fuzzy_system_control = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5, rule6, rule7])
        fuzzy_system = ctrl.ControlSystemSimulation(fuzzy_system_control)

        # Run the system with Polarity and Subjectivity
        try:
            fuzzy_system.input['Polarity'] = avg_Polarity
            fuzzy_system.input['Subjectivity'] = avg_Subjectivity
            
            fuzzy_system.compute()
        
            strength = fuzzy_system.output['strength']
        except:
            strength = 0

        return strength

    def tick(self):
        self.lock.acquire()
        self.log.info('Beginning news updates')

        end = (datetime.datetime.now() + datetime.timedelta(days=0)).strftime("%Y-%m-%dT%H:%M:%SZ")
        start = (datetime.datetime.now() + datetime.timedelta(days=-1)).strftime("%Y-%m-%dT%H:%M:%SZ")

        news_dump = Broker.api.get_news("BTCUSD", start, end,exclude_contentless="True",include_content="True",limit=50)

        News_cols = ['date','time','link','polarity_content','subjectivity_content','polarity_title','subjectivity_title']
        news = pd.DataFrame(columns = News_cols)

        for j in range(len(news_dump)):
            soup_content = BeautifulSoup(news_dump[j].content)
            doc_content = nlp(soup_content.get_text())
            polarity_content = doc_content._.blob.polarity
            subjectivity_content = doc_content._.blob.subjectivity

            soup_title = BeautifulSoup(news_dump[j].headline)
            doc_title = nlp(soup_title.get_text())
            polarity_title = doc_title._.blob.polarity
            subjectivity_title = doc_title._.blob.subjectivity

            time_news = news_dump[j].created_at
                    
            link = news_dump[j].url

            news.loc[len(news)] = [start, time_news, link, polarity_content, subjectivity_content, polarity_title, subjectivity_title]

        df = news.copy()
        df['strength_content'] = 0
        df['strength_title'] = 0
            
        for i,row in df.iterrows():
            df.at[i,'strength_content'] = self.News_Fuzzy(row['polarity_content'], row['subjectivity_content'])
            df.at[i,'strength_title'] = self.News_Fuzzy(row['polarity_title'], row['subjectivity_title'])

        # df.hist('strength_content',bins=50)
        # df.hist('strength_title',bins=50)

        df['news_type'] = 0
        for i,row in df.iterrows():
            if row['strength_content'] > 0 and row['strength_title'] > 0:
                df.at[i, 'news_type'] = 1
            elif row['strength_content'] < 0 and row['strength_title'] < 0:
                df.at[i, 'news_type'] = -1
            else:
                df.at[i, 'news_type'] = 0
        df['EWM_strength'] = df['news_type'].ewm(span=len(df),adjust=False).mean()
        self.current_strength = df.iloc[-1].EWM_strength

        self.log.info('news updates')
        self.log.info(df)
        self.lock.release()

    def loop(self):
        while True:
            self.tick()
            time.sleep(self.tick_time)
        
    def peek(self):
        self.log.info(self.current_strength)
        return self.current_strength

    def __str__(self):
        return 'News_Reviewer'

# Reviewer
- Get Signals and use the knowledge class agent weights to send the importance of Buy, Sell and Hold Signals

In [17]:
class Reviewer():

    def __init__(self, knowledge):
        self.lock = Lock()
        self.log = logging.getLogger('Reviewer')
        self.log.info("Initialized Reviewer")
        self.knowledge = knowledge

    def review(self, entry):
        # Find all 5 signals and knowledge to find action
        # Entry rows = SMA, EMA, BB, BBT, FBP, price

        buy_importance = 0
        sell_importance = 0
        hold_importance = 0

        agent_weights = self.knowledge.dump_agent_weights()
        for i in ['SMA','EMA','BB','BBT','FBP']:
            if entry[i] == 1:
                buy_importance = buy_importance + agent_weights[i][1]
                self.log.info(("BUY",i,agent_weights[i][1]))
            elif entry[i] == -1:
                sell_importance = sell_importance + agent_weights[i][-1]
                self.log.info(("SELL",i,agent_weights[i][-1]))
            else:
                hold_importance = hold_importance + agent_weights[i][0]
                self.log.info(("HOLD",i,agent_weights[i][0]))
        self.log.info((buy_importance, sell_importance, hold_importance))

        return [buy_importance, sell_importance, hold_importance ]


    def __str__(self):
        return 'Reviewer'

# CEO

The Agent to control them all.

CEO has access to all other agents except learner.

The main function of the agent is tick.

It ticks every tick_time (1 minute)

The following is the operation the CEO does eevry tick
- Get latest signals for the signal agent
- Call reviwer to get Buy,Sell,Hold weights
- Use Risk Level to decide on final action. Deafult is 2
    - Risk Level 1,2,3 = Limit Orders
    - Risk Level 4,5,6 = Market Orders
    - Risk Level 1,4 = Need absolute dominance of a Buy/Sell signal over the other 2 signals to actually place an order
    - Risk Level 2,5 = As long as Buy>Sell and both of them combined are greater than hold, we buy
    - Risk Level 3,6 = Try to Buy/Sell as oftern as possible
    - Risk ramps up quite fast as soon as you increae the numbers.
- Use news outlook to decide if we should act on a signal
    - If news outlook is at the extremes, we don;t trade and instead wait for human intervention
- If action is a Buy or a Sell
    - Use case Based Reasoning to decide on a CBR multiplier
        - Go to all previous cases where such a signla composition existed
        - FInd number of profitable trades, bad trades, and neutral trades
        - CBR_multiplier is calculated as 
            - 1 + ((good trades - bad trades)/(good+bad_neutral))
            - example : assume for a configuration, we have 2 good, 3 bad and 0 neutrals
                - we find CBR multiplier as 1 + (2-3)/5) => 0.8
                - we trade 0.8 of the default volume in such a case
    - Also use Market Trend from Bollinger Band Trends to evaluate a Trend Multiplier
        - For a Buy signal, if the market trend is very good, we increase the volume to twice the default.
        - Exact details in code below
    - We then calculate actual volume we are going to trade.
    - We then look at total volume of the market. If less than 1 BTC is being traded in the past minute, we opt to not trade.
    - We then set stoploss and take profits
    - We then check if we have the crypto/cash balance and then place the trade.
    - If the order is a limit order, we use Limit_Buy_Sell_Cost_variance to increase the price of a buy or decrese the price of a sell. We do this so that we can actually fill our order. Giving the exact price sometimes does not allow us to fill the order even after a long time.
        - In an ideal world, this variance should be 0.

- After performing the action, at the end of each tick, we compile the latest actions, wallent, current PnL, and other things. We also monitor a few values for the CEO
    - Total Assets
        - If total assets fall below 90k, it is a Warning
        - If total assets fall below 80k, it is a Danger
        - Else Normal
    - Cash
        - If total assets fall below 60k, it is a Warning
        - If total assets fall below 50k, it is a Danger
        - Else Normal
    - News Outlook
        - If overall News outlook is >0.6 or <-0.6, then its a warning
        - IF overall News outlook is >0.9 or <-0.9, then its a Danger
        - Else Normal
- We use the above parameters to decide on the current status of the CEO and log it with the host of other data
- We also send them to PowerBI is the relavant setup is done
- Human interaction is expected when Level is WARNING and required when level is DANGER

The CEO agent also has a set of human interaction options to tweak the system on the fly
- Change Risk Level
- Force an order
    - Give volume, price and action to force an order. It will still be recorded and analysed
- Pause or Resume the system
- Sell all crypto, cancel all orders and shutdown

Human interactions is explained in detail in the following section

In [18]:
class CEO():

    def __init__(self, knowledge, order, signals, reviewer, news_reviewer):
        self.lock = Lock()
        self.tick_time = tick_time
        self.log = logging.getLogger('CEO')
        self.log.info("Initialized CEO")
        self.knowledge = knowledge
        self.order = order
        self.signals = signals
        self.reviewer = reviewer
        self.news_reviewer = news_reviewer
        self.risk_level = 2
        self.status = True

        self.wallet_index = []
        self.orders_index = []
        self.order_profits_index = []

        time.sleep(10)

        self.thread = Thread(name=self.__str__(), target=self.loop)
        self.thread.start()

    def loop(self):
        while self.status:
            self.tick()
            time.sleep(self.tick_time)
										
    def tick(self):
        self.lock.acquire()
        self.log.info("CEO TICK START")
        
        signals = self.signals.peek()
        self.log.info("Signals Recieved")
        self.log.info(("SIGNALS ",list(signals[['SMA','EMA','BB','BBT','FBP','BBT_val']])))

        # DECIDER START
        # Review for Buy,Sell,Hold
        [buy,sell,hold] = self.reviewer.review(signals)
        self.log.info(("buy,sell,hold",[buy,sell,hold]))
        risk_level = self.risk_level
        action = 'hold'
        if risk_level == 1 or risk_level == 4:
            if buy >= (sell+hold):
                action = 'buy'
            elif sell >= (buy+hold):
                action = 'sell'
            else:
                action = 'hold'
        elif risk_level == 2 or risk_level == 5:
            if (hold > buy) and (hold > sell):
                action = 'hold'
            elif buy > sell:
                action = 'buy'
            elif sell > buy:
                action = 'sell'
        elif risk_level == 3 or risk_level == 6:
            if buy > sell:
                action = 'buy'
            elif sell > buy:
                action = 'sell'
            else:
                action = 'hold'

        self.log.info(("Risk and Action ",risk_level,action))

        # news reviewer
        news_outlook = float(self.news_reviewer.peek())

        if news_outlook >= -0.9 and news_outlook <= 0.9:

            if action == 'hold':
                self.log.info(("HOLD AT ",time.time()))
            else:
                # CBR 
                order_profitability = pd.DataFrame(self.order.order_profits)
                # print(order_profitability)
                temp_df = order_profitability.loc[(order_profitability['SMA']==signals.SMA) & (order_profitability['EMA']==signals.EMA) & 
                                (order_profitability['BB']==signals.BB) & (order_profitability['BBT']==signals.BBT) &
                                (order_profitability['FBP']==signals.FBP) ] # & ( (abs(order_profitability['BBT_val'])/order_profitability['BBT_val']) == (abs(signals.BBT_val)/signals.BBT_val) )
                
                pos = len(temp_df.loc[temp_df['profit_ticks_5'] > 0]) + 1
                neg = len(temp_df.loc[temp_df['profit_ticks_5'] < 0]) + 1
                neutral = len(temp_df.loc[temp_df['profit_ticks_5'] == 0]) + 1
                CBR_score = (pos-neg)/(pos+neg+neutral)
                self.log.info(("CBR ",CBR_score))
                    
                # Trend Analysis
                trend_multiplier = 1
                if action == 'buy':
                    if signals['BBT_val'] >= 0.2:
                        trend_multiplier = 2
                    elif signals['BBT_val'] >= 0.1:
                        trend_multiplier = 1.5
                    elif signals['BBT_val'] > 0:
                        trend_multiplier = 1.1
                    elif signals['BBT_val'] == 0:
                        trend_multiplier = 0
                    elif signals['BBT_val'] <= -0.2:
                        trend_multiplier = 0.1
                    elif signals['BBT_val'] <= -0.1:
                        trend_multiplier = 0.25
                    elif signals['BBT_val'] < 0:
                        trend_multiplier = 0.5
                elif action == 'sell':
                    if (-1*signals['BBT_val']) >= 0.2:
                        trend_multiplier = 2
                    elif (-1*signals['BBT_val']) >= 0.1:
                        trend_multiplier = 1.5
                    elif (-1*signals['BBT_val']) > 0:
                        trend_multiplier = 1.1
                    elif (-1*signals['BBT_val']) == 0:
                        trend_multiplier = 0
                    elif (-1*signals['BBT_val']) <= -0.2:
                        trend_multiplier = 0.1
                    elif (-1*signals['BBT_val']) <= -0.1:
                        trend_multiplier = 0.25
                    elif (-1*signals['BBT_val']) < 0:
                        trend_multiplier = 0.5
                self.log.info(("trend and multiplier",signals['BBT_val'] ,trend_multiplier))

                order_type = 'limit'
                if risk_level >= 4:
                    order_type = 'market'
                self.log.info(("order type ", order_type))

                default_volume = 0.01
                volume = default_volume * (1+CBR_score) * trend_multiplier

                volume = round(volume, 4)
                self.log.info(('default_volume ',default_volume))
                self.log.info(("vol ",volume))

                price = Broker.get_ticker_price()

                if action =='buy':
                    price = price + Limit_Buy_Sell_Cost_variance
                elif action =='sell':
                    price = price - Limit_Buy_Sell_Cost_variance

                # StopLoss and takeProfit
                stop_loss = price *0.9
                take_profit = price * 1.05
                self.log.info(("Price, StopLoss, TakeProfit ",price,stop_loss,take_profit))

                SMA = signals.SMA
                EMA = signals.EMA
                BB = signals.BB
                BBT = signals.BBT
                FBP = signals.FBP
                BBT_val = signals.BBT_val

                
                # DECIDER END
                

                if float(signals.volume) > 1:
                    self.log.info('market liquidity present')
                    self.log.info((action, 'limit', volume, price, 
                            stop_loss, take_profit, EMA, SMA, BB, BBT, FBP, BBT_val))
                    # print((action, 'limit', volume, price, 
                    #         stop_loss, take_profit, EMA, SMA, BB, BBT, FBP, BBT_val))
                    self.log.info("PLACING ORDER")
                    
                    crypto = 0
                    if len(Broker.get_crypto_balance()) > 0:
                        crypto = Broker.get_crypto_balance()[symbol]
                    cash = Broker.get_balance()
                    if action == 'buy':
                        if float(cash) >= float(price)*float(volume):
                            self.order.place_order(action = action, order_type = order_type, volume = volume, price = price, 
                            stop_loss = stop_loss, take_profit = take_profit, EMA = EMA, SMA = SMA, BB = BB, BBT = BBT, FBP = FBP, BBT_val = BBT_val)
                        else:
                            self.log.info("NO BALANCE, CAN'T BUY")
                    elif action == "sell":
                        if float(crypto) >= float(volume):
                            self.order.place_order(action = action, order_type = order_type, volume = volume, price = price, 
                            stop_loss = stop_loss, take_profit = take_profit, EMA = EMA, SMA = SMA, BB = BB, BBT = BBT, FBP = FBP, BBT_val = BBT_val)
                        else:
                            self.log.info("NO CRYPTO, CAN'T SELL")
        else:
            self.log.info("very risky market. holding all sales. waiting for human instructions")


        self.log.info('Beginning powerBI send')

        # PowerBI Wallet
        wallet = self.order.wallet.copy()
        sent_index_wallet = self.wallet_index
        all_index_wallet = list(wallet.index)
        to_send_wallet = list(set(all_index_wallet)-set(sent_index_wallet))

        orders = self.order.orders.copy()
        num_total_orders = len(orders)
        num_buy_orders = len(orders.loc[orders['action'] == 'buy'])
        num_sell_orders = len(orders.loc[orders['action'] == 'sell'])

        # PowerBI CEO
        Status = 'Normal'
        Desc = 'Normal'
        time_ceo = datetime.datetime.utcfromtimestamp(time.time()).strftime("%Y-%m-%d %H:%M:%S")
        acc_dets = Broker.get_account()
        cash = float(acc_dets.cash)
        total_assets = float(acc_dets.equity)

        Status1 = 1
        if total_assets < 80000:
            Status1 = 3 # Danger
            Desc = Desc +'[Total Equity < 80k]'
        elif total_assets < 90000:
            Status1 = 2 # warn
            Desc = Desc +'[Total Equity < 90k]'

        Status2 = 1
        if cash < 50000:
            Status2 = 3 # Danger
            Desc = Desc +'[Cash < 50k]'
        elif total_assets < 60000:
            Status2 = 2 # warn
            Desc = Desc +'[Cash < 60k]'

        Status3 = 1
        if news_outlook <= -0.8 or news_outlook >= 0.8:
            Status3 = 3 # Danger
            Desc = Desc +'[News Outlook at the extremes]'
        elif news_outlook <= -0.6 or news_outlook >= 0.6:
            Status3 = 2 # Warn
            Desc = Desc +'[News Outlook approcahing extremes]'

        Status_val = max(Status1,Status2,Status3)
        if Status_val == 1:
            Status = 'Normal'
        elif Status_val == 2:
            Status = 'Warning'
        elif Status_val == 3:
            Status = 'Danger'

        self.log.info("PowerBI and CEO Status")
        self.log.info(to_send_wallet)
        for i in to_send_wallet:
            
            to_send_status = wallet.iloc[i].to_dict()
            to_send_status['time'] = to_send_status['time']#.replace("T",' ').replace('Z','')
            to_send_status['num_buy'] = num_buy_orders
            to_send_status['num_sell'] = num_sell_orders
            to_send_status['num_total'] = num_total_orders
            to_send_status['CEO_status'] = Status
            to_send_status['CEO_desc'] = Desc

            to_send_status['cash'] = float(to_send_status['cash'])
            to_send_status['crypto'] = float(to_send_status['crypto'])
            to_send_status['crypto_value'] = float(to_send_status['crypto_value'])
            to_send_status['total_assets'] = float(to_send_status['total_assets'])
            to_send_status['PnL'] = float(to_send_status['PnL'])
            to_send_status['crypto_price'] = float(to_send_status['crypto_price'])
            to_send_status['risk_level'] = float(self.risk_level)

            self.log.info([to_send_status])

            if POWERBI_USED :
                self.log.info("SEND TO POWERBI")
                headers = {
                "Content-Type": "application/json"
                }
                response = requests.request(
                    method="POST",
                    url=POWERBI_DATA_URL,
                    headers=headers,
                    data=json.dumps([to_send_status])
                )
                self.log.info(response)
                self.log.info("SENT TO POWERBI")
            
        self.wallet_index = all_index_wallet
        self.log.info("CEO status update done")


        self.log.info('powerBI sends finished')


        self.log.info("CEO TICK ENDED")
        self.lock.release()

    def change_risk_level(self, new_risk):
        self.lock.acquire()
        self.log.info("changing risk")
        self.log.info(["OLD RISK",self.risk_level])
        self.risk_level = new_risk
        self.log.info(["NEW RISK",self.risk_level])
        self.log.info("changed risk")
        self.lock.release()

    def change_status(self, status):
        self.lock.acquire()
        self.log.info("changing status")
        self.log.info(["OLD status",self.status])
        self.status = status
        self.log.info(["NEW status",self.status])
        self.log.info("changed status")
        self.lock.release()

    def get_out_of_the_game(self):
        self.lock.acquire()
        self.log.info("selling all crypto")
        Broker.cancel_all_orders()
        crypto = 0
        if len(Broker.get_crypto_balance()) > 0:
            crypto = Broker.get_crypto_balance()[symbol]
        Broker.place_market_sell_order(symbol, float(crypto), str(time.time()))
        self.log.info("sold all crypto")
        self.log.info("Stopping functioning")
        self.status = False
        self.lock.release()

    def force_orders(self, action, order_type, volume, price, stop_loss, take_profit):
        self.lock.acquire()
        signals = self.signals.peek()
        SMA = signals.SMA
        EMA = signals.EMA
        BB = signals.BB
        BBT = signals.BBT
        FBP = signals.FBP
        BBT_val = signals.BBT_val
        self.log.info((action, 'limit', volume, price, 
                            stop_loss, take_profit, EMA, SMA, BB, BBT, FBP, BBT_val))
        self.log.info("PLACING FORCED ORDER")
                    
        crypto = 0
        if len(Broker.get_crypto_balance()) > 0:
            crypto = Broker.get_crypto_balance()[symbol]
        cash = Broker.get_balance()
        if action == 'buy':
            if float(cash) >= float(price)*float(volume):
                self.order.place_order(action = action, order_type = order_type, volume = volume, price = price, 
                stop_loss = stop_loss, take_profit = take_profit, EMA = EMA, SMA = SMA, BB = BB, BBT = BBT, FBP = FBP, BBT_val = BBT_val)
            else:
                self.log.info("NO BALANCE, CAN'T BUY")
        elif action == "sell":
            if float(crypto) >= float(volume):
                self.order.place_order(action = action, order_type = order_type, volume = volume, price = price, 
                stop_loss = stop_loss, take_profit = take_profit, EMA = EMA, SMA = SMA, BB = BB, BBT = BBT, FBP = FBP, BBT_val = BBT_val)
            else:
                self.log.info("NO CRYPTO, CAN'T SELL")
        self.lock.release()
    def __str__(self):
        return 'CEO'

# GOD
Overall Class to start all agents 

Initializes each agent with required parameters

In [19]:
class God(object):
    def __init__(self):
        # Continuously Generate Signals
        self.Signals = Signals(SMA(),EMA(),BB(),BBT(),FBP())###

        time.sleep(10)

        # Record Orders. Update Orders. Monitor Orders
        self.Orders = Orders(self.Signals)###

        # Initialize Signal Weights. Update weights, etc
        self.Knowledge = Knowledge()###

        # Check if trade is possible and decide on stoploss/takeprofit
        self.Reviewer = Reviewer(self.Knowledge)

        # Periodically, virtually sell all. Then update Knowledge weights
        self.Learner = Learner(self.Knowledge , self.Orders)

        # Alpaca Benzinga News and Fuzzy Logic to find results
        self.News_Reviewer = News_Reviewer()

        # Use signals and weights to decide on action. then use CBR and Risk Appetite to decide volume. 
        # Review trade to see if it is possible and decide on stoploss/takeprofit. Then place limit order. 
        self.CEO = CEO(self.Knowledge ,self.Orders, self.Signals, self.Reviewer, self.News_Reviewer)
        
        # send trades to powerbi
        # self.Visulalizer = Visualizer(self.Knowledge, self.Orders)

    # def change_risk_appetite(self, new_val):
    #     self.RiskAppetite.change_risk_appetite(new_val)

# Start System

In [20]:
G = God()
print("THE SYSTEM HAS BEGUN RUNNING")
print()
print("DO NOT QUIT THE COLAB SESSIONS> TRADING IS HAPPENING IN BACKGROUND THREADS")
print("TO STOP THE SYSTEM, CLICK FACTORY RESeT RUNTIME UNDER RUNTIME OR CLOSE THE BROSWER TAB")
print("DATA WILL BE SAVED IN THE GOOGLE DRIVE FOLDER WHERE YOU GAVE A PATH")

THE SYSTEM HAS BEGUN RUNNING

DO NOT QUIT THE COLAB SESSIONS> TRADING IS HAPPENING IN BACKGROUND THREADS
TO STOP THE SYSTEM, CLICK FACTORY RESeT RUNTIME UNDER RUNTIME OR CLOSE THE BROSWER TAB
DATA WILL BE SAVED IN THE GOOGLE DRIVE FOLDER WHERE YOU GAVE A PATH


# Human Interactions

There are a host of controls a human can use to control and tweak the algorithm as they run. All the currently implemented controls are found below with more on the way

## Change Risk Level
uncomment the below code and set preferred risk level to change the risk level the system operates in


In [21]:
# G.CEO.change_risk_level(1)

## Change Status
Uncomment the relavant part of the below cell to either Pause or Resume the system

In [22]:
# # Pause the system
# G.CEO.change_status(False)

# #Resume the system
# G.CEO.change_status(True)

## Force an order
Set the relavant variables to force the system to execute a buy/sell order

In [23]:
# G.CEO.force_orders(action = 'buy', order_type='market', volume=0.01, price=50000, stop_loss=40000, take_profit=60000)

## Pull the plug
Sell all crypto and end the system

In [24]:
# G.CEO.get_out_of_the_game()

# Code to view last entry of wallet
Rerun whwnever to view the last entry to wallet

In [25]:
G.CEO.order.wallet.tail(1)

,time,cash,crypto,crypto_value,num_open_orders,num_orders,total_assets,PnL,crypto_price
0,2022-04-10T12:10:10Z,97823.9085,0.2942,12554.278037,0,50,110378.186537,10378.186537,42836.7


# Conclusion
- The system generates a profit at times and loss at others
- The sytem is too simplistic for the current market
- A lot of potetial improvements exist that could have been done if we had time



- Pnl Results over a long period of time
        BTC PRICE = $42,319.50	
        CRYPTO = 0.2442	
        CRYPTO VAL = $10,303.30
        CASH = $99,439.95
        EQUITY = $109,747.14

        April 5 00:30 AM to April 9 11 AM

        System was restarted around 17 times and run over 50 differen time periods.
        Restart refers to dleeting agent weights, orders and order profits tables. The trades have happened in Alpaca but not used by the system anymore

        We got lucky with BTC price increases and not all trials have been this successfull.

        Worst luck : 
            April 3. BTC Market buy happened at close to 400$ over market price.
            April 5. BTC Market sell happened at close to 300$ below market price

# Possible Extensions
- get all historical Trades from Alpaca API and use then instead of using a separate file
    - Can help maintain system history without complications
    - Currently, some limit orders can just be left hanging in Alpaca if system is shut down at a bad time
- Define a better order prfitability measure
- Implement more advanced signals
    - MACD and the likes for staying on the statistical side
    - Facebook Prophet, Tensorflow Timeseries, etc fr ML based signals
- Use Fuzzy Logic for more advanced decisions
    - Example 1 : Use current PnL from wallet, news and CBR with Fuzzy Logic to dynamically change Risk Level
    - Example 2 : Use Fuzzy Logic to decide when to warn the CEO instead of using limits
- Code can be more modular and have much less hard_coded variables
- Case Based Reasoning could be more advanced
    - Can be used to boost buy/sell/hold weights during learning or reviewing
    - Can implement ML based methods too
- Tick Time minimum is 1 Minte, but High frequency trading is much faster. System could use better algorithms to tick and operate faster.

# Contact Us

If something is still confusing with the code, it crashes, or you wanna just chat. feel free to email any of the authors